In [1]:
import clip
print(clip.__file__)
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
from multiproc_converter_to_png import convert_svgs_to_pngs_parallel, load_images_parallel
from tqdm.notebook import tqdm
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
import os
from tifa.tifascore import tifa_score_benchmark
from configs.config_util import *
import json
cur_path = os.getcwd()
cur_path

/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/clip/__init__.py


/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/fairseq/modules/multihead_attention.py:15: FutureWarning: xformers.components is deprecated and is not maintained anymore. It might be removed in a future version of xFormers 
  from xformers.components.attention import build_attention
/home/student/kuzin/py

'/home/student/kuzin/python_project/TextToSvg_MLCUP'

In [2]:
config_path = os.path.join(cur_path, 'configs',"Qwen3_0_6B.yaml")
print(config_path)
config = load_config(config_path)
print(config)

/home/student/kuzin/python_project/TextToSvg_MLCUP/configs/Qwen3_0_6B.yaml
ModelConfig(path='Qwen/Qwen3-0.6B', max_new_tokens=1500, enable_thinking=False, torch_dtype='bfloat16', device_map='cuda', attn_implementation='flash_attention_2')


In [3]:
model = AutoModelForCausalLM.from_pretrained(config.path, device_map = config.device_map, torch_dtype = config.torch_dtype,attn_implementation = config.attn_implementation)
tokenizer = AutoTokenizer.from_pretrained(config.path)
model.bfloat16()
tokenizer.padding_side  = 'left'

### Sanity check correct model upload

In [4]:
messages = [
 {"role": "user", "content": "Generate SVG image from description: 'On a gray day a surfer carrying a white board walks on a beach' .The SVG scene should be of the following dimensions 256*256. First, think about the scene, about the objects on the stage and their characteristics. Draw simple. Make sure that all tags are closed and the svg is correct!"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    repetition_penalty=10,  # немного штрафует за повторы
    enable_thinking = config.enable_thinking, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = config.max_new_tokens, # Increase for longer outputs!
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Sure! Below is an SVG image based on the description:

```xml
<svg width="256" height="256" xmlns="http://www.w3.org/2000/svg">
  <!-- Background (gray) -->
  <rect width="256" height="256" fill="gray" />

  <!-- Surfer and white board -->
  <rect x="100" y="100" width="100" height="100" fill="white" />
</svg>
```

### Notes:
- The SVG is 256x256 pixels.
- The surfer is centered at `(100, 100)`.
- The white board is drawn as a rectangle of 100x100 pixels.
- The background is a gray rectangle covering the entire canvas.

Let me know if you'd like to add more details!<|im_end|>


In [5]:
### want
"""
data/
    Qwen/
        model_name/
            tifa_generation.json : {"coco_301091": "./src/coco_301091.jpg", ..., }
            src/
                coco_301091.jpg
    tifa_eval_dataset.csv : {"coco_301091": "Desciption in words", ..., }
"""


'\ndata/\n    Qwen/\n        model_name/\n            tifa_generation.json : {"coco_301091": "./src/coco_301091.jpg", ..., }\n            src/\n                coco_301091.jpg\n    tifa_eval_dataset.csv : {"coco_301091": "Desciption in words", ..., }\n'

### Create  tifa_eval_dataset.csv

In [6]:
import pandas as pd
df = pd.read_json('./tifa/tifa_v1.0/tifa_v1.0_text_inputs.json')
df.to_csv("./data/tifa_eval_dataset.csv")
eval_dataset = df

In [7]:
import pandas as pd
df = pd.read_json('./tifa/tifa_v1.0/tifa_v1.0_text_inputs.json')
print(f"Количество записей: {len(df)}")
print(f"Колонки: {df.columns.tolist()}")
print("\nПервые 5 строк:")
df.head()

Количество записей: 4081
Колонки: ['id', 'caption', 'coco_val_id']

Первые 5 строк:


,id,caption,coco_val_id
0,coco_301091,On a gray day a surfer carrying a white board ...,380711.0
1,coco_421933,A person sitting on a horse in air over gate i...,260106.0
2,coco_55404,"A small room with a futon couch, a sewing mach...",39477.0
3,coco_738534,A pile of orange sitting in a blue bowl.,290081.0
4,coco_788027,A horse and several cows feed on hay.,61171.0


In [8]:
optional_prompt = "The SVG scene should be of the following dimensions 256*256. First, think about the scene, about the objects on the stage and their characteristics. Make sure that all tags are closed and the svg is correct! Answer is only SVG without comments!"
def get_sample(row):
    user_content_prompt = f"Generate SVG image from description <{row.caption[:-1]}>. {optional_prompt}"
    return [{"role": "user", "content": user_content_prompt}]


eval_dataset["eval_data"] = eval_dataset.apply(get_sample, axis=1)
eval_dataset.eval_data[1:3]

1    [{'role': 'user', 'content': 'Generate SVG ima...
2    [{'role': 'user', 'content': 'Generate SVG ima...
Name: eval_data, dtype: object

In [9]:
dataset = Dataset.from_dict({"query": list(eval_dataset["eval_data"])})
dataset[1]['query']

[{'content': 'Generate SVG image from description <A person sitting on a horse in air over gate in grass with people and trees in background>. The SVG scene should be of the following dimensions 256*256. First, think about the scene, about the objects on the stage and their characteristics. Make sure that all tags are closed and the svg is correct! Answer is only SVG without comments!',
  'role': 'user'}]

In [10]:
dataset = dataset.map(lambda x: {"formatted_query": tokenizer.apply_chat_template(x["query"], tokenize=False, add_generation_prompt=True, enable_thinking=False)}, remove_columns=["query"])

Map:   0%|          | 0/4081 [00:00<?, ? examples/s]

In [11]:
print(dataset['formatted_query'][0])

<|im_start|>user
Generate SVG image from description <On a gray day a surfer carrying a white board walks on a beach>. The SVG scene should be of the following dimensions 256*256. First, think about the scene, about the objects on the stage and their characteristics. Make sure that all tags are closed and the svg is correct! Answer is only SVG without comments!<|im_end|>
<|im_start|>assistant
<think>

</think>




In [12]:
plug_prompt =  "Okay, I think I have finished thinking"
import re

def insert_text_between_think_tags(original_text):
    original_text = original_text["formatted_query"]
    return {
        "formatted_query":
        re.sub(
        r'(<think>).*?(</think>)',
        r'\1{}\2'.format(plug_prompt),
        original_text,
        flags=re.DOTALL
    )
    }
dataset = dataset.map(insert_text_between_think_tags)
print(dataset), dataset[0]

Map:   0%|          | 0/4081 [00:00<?, ? examples/s]

Dataset({
    features: ['formatted_query'],
    num_rows: 4081
})


(None,
 {'formatted_query': '<|im_start|>user\nGenerate SVG image from description <On a gray day a surfer carrying a white board walks on a beach>. The SVG scene should be of the following dimensions 256*256. First, think about the scene, about the objects on the stage and their characteristics. Make sure that all tags are closed and the svg is correct! Answer is only SVG without comments!<|im_end|>\n<|im_start|>assistant\n<think>Okay, I think I have finished thinking</think>\n\n'})

In [13]:
def preprocess_function(examples):
    return tokenizer(examples["formatted_query"], padding=False, truncation=False)
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=["formatted_query"]
)

Map (num_proc=4):   0%|          | 0/4081 [00:00<?, ? examples/s]

In [14]:
print(tokenizer.batch_decode(tokenized_dataset[0]['input_ids'], skip_special_tokens=True))

['', 'user', '\n', 'Generate', ' SVG', ' image', ' from', ' description', ' <', 'On', ' a', ' gray', ' day', ' a', ' sur', 'fer', ' carrying', ' a', ' white', ' board', ' walks', ' on', ' a', ' beach', '>.', ' The', ' SVG', ' scene', ' should', ' be', ' of', ' the', ' following', ' dimensions', ' ', '2', '5', '6', '*', '2', '5', '6', '.', ' First', ',', ' think', ' about', ' the', ' scene', ',', ' about', ' the', ' objects', ' on', ' the', ' stage', ' and', ' their', ' characteristics', '.', ' Make', ' sure', ' that', ' all', ' tags', ' are', ' closed', ' and', ' the', ' svg', ' is', ' correct', '!', ' Answer', ' is', ' only', ' SVG', ' without', ' comments', '!', '', '\n', '', 'assistant', '\n', '<think>', 'Okay', ',', ' I', ' think', ' I', ' have', ' finished', ' thinking', '</think>', '\n\n']


In [15]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Создаем DataLoader
dataloader = DataLoader(
    tokenized_dataset,
    batch_size=64,
    shuffle=False,
    collate_fn=data_collator,
)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [16]:
once_checked= False
svg_answers = []
for batch in tqdm(dataloader, desc="Генерация SVG"):
    batch = {k: v.to("cuda") for k, v in batch.items()}
    with torch.no_grad():
        print()
        outputs = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_new_tokens=config.max_new_tokens,
            use_cache = True,
            length_penalty=1.1,
        )
        input_ids = batch["input_ids"].cpu()
        outputs =  outputs.cpu()
        batch_output_ids = [
            outputs[i][len(input_ids[i]):].tolist()
            for i in range(input_ids.size(0))
        ]
    decoded_responses= tokenizer.batch_decode(batch_output_ids, skip_special_tokens=True)
    if once_checked == False:
        once_checked = True
        print("simple check")
        print(decoded_responses)
    svg_answers+=decoded_responses

Генерация SVG:   0%|          | 0/64 [00:00<?, ?it/s]


simple check
['<svg width="256" height="256" xmlns="http://www.w3.org/2000/svg">\n  <!-- The surfer and the white board -->\n  <rect x="100" y="100" width="156" height="156" fill="white" />\n  <rect x="200" y="100" width="156" height="156" fill="gray" />\n  <rect x="150" y="150" width="100" height="100" fill="white" />\n  <rect x="150" y="150" width="100" height="100" fill="black" />\n</svg>', '<svg width="256" height="256" xmlns="http://www.w3.org/2000/svg">\n  <rect width="256" height="256" fill="white"/>\n  <polygon points="128,0 192,128 192,224 0,224 0,0" fill="white"/>\n  <rect width="128" height="224" fill="white"/>\n  <polygon points="128,0 192,128 192,224 0,224 0,0" fill="white"/>\n  <circle cx="128" cy="128" r="10"/>\n  <circle cx="192" cy="128" r="10"/>\n  <circle cx="192" cy="224" r="10"/>\n  <circle cx="0" cy="224" r="10"/>\n  <circle cx="0" cy="0" r="10"/>\n  <path d="M 128 0 L 128 224" fill="white"/>\n  <path d="M 192 0 L 192 224" fill="white"/>\n  <path d="M 0 128 L 0 2

In [32]:
dir_path_for_res = os.path.join(cur_path,"data", config.path)
print(dir_path_for_res)
file_path = os.path.join(dir_path_for_res, "tifa_outputs_model.csv")

/home/student/kuzin/python_project/TextToSvg_MLCUP/data/Qwen/Qwen3-0.6B


In [18]:
os.makedirs(os.path.dirname(file_path), exist_ok=True)
temp = pd.DataFrame(svg_answers)
temp.to_csv(file_path,index = False)

In [19]:
dir_path_for_res

'/home/student/kuzin/python_project/TextToSvg_MLCUP/data/Qwen/Qwen3-0.6B'

In [20]:
output_model = pd.read_csv(file_path)
eval_dataset["svg_code"] = output_model
eval_dataset["wrong_gen"] = 1

In [21]:
output_folder = os.path.join(dir_path_for_res, "src")

In [22]:
for_path_columns = []
num_processes= 32
for i in tqdm(range(0, len(eval_dataset), num_processes), 
              total=len(range(0, len(eval_dataset), num_processes)), 
              desc="Processing batches"):
    start_i = i
    end_i = i+ num_processes
    batch = eval_dataset.iloc[start_i:end_i].set_index('id')['svg_code'].to_dict()
    res = convert_svgs_to_pngs_parallel(batch,output_folder,num_processes)
    # res return {'coco_301091': True, 'coco_421933': False} True if correct gen
    mapping = {k: (1 if v else 0) for k, v in res.items()}
    
    # Обновляем только те строки, где caption есть в res
    eval_dataset.loc[eval_dataset['id'].isin(mapping.keys()), 'wrong_gen'] = \
        eval_dataset['id'].map(mapping)

Processing batches:   0%|          | 0/128 [00:00<?, ?it/s]

Запуск обработки на 32 процессах...
Заменяю на заглушку coco_317050: unclosed token: line 45, column 2
Запуск обработки на 32 процессах...
Запуск обработки на 32 процессах...
Заменяю на заглушку coco_461808: unclosed token: line 56, column 2
Заменяю на заглушку coco_148563: unclosed token: line 75, column 2
Запуск обработки на 32 процессах...
Заменяю на заглушку coco_379749: unclosed token: line 54, column 4
Запуск обработки на 32 процессах...
Заменяю на заглушку coco_182098: unclosed token: line 40, column 2
Запуск обработки на 32 процессах...
Запуск обработки на 32 процессах...
Заменяю на заглушку coco_579942: no element found: line 53, column 0Заменяю на заглушку coco_381963: unclosed token: line 48, column 2

Запуск обработки на 32 процессах...
Заменяю на заглушку coco_492798: unclosed token: line 48, column 2
Заменяю на заглушку coco_637420: unclosed token: line 53, column 2Заменяю на заглушку coco_55863: unclosed token: line 79, column 2

Запуск обработки на 32 процессах...
Замен

In [23]:
eval_dataset["paths"] = eval_dataset.apply(lambda sample: {sample.id: os.path.join(output_folder, f"{sample.id}.png" )}  ,axis=1)

In [24]:
eval_dataset.paths.head(5)

0    {'coco_301091': '/home/student/kuzin/python_pr...
1    {'coco_421933': '/home/student/kuzin/python_pr...
2    {'coco_55404': '/home/student/kuzin/python_pro...
3    {'coco_738534': '/home/student/kuzin/python_pr...
4    {'coco_788027': '/home/student/kuzin/python_pr...
Name: paths, dtype: object

In [25]:
len(eval_dataset[eval_dataset.wrong_gen==1])

3927

In [26]:
merged_dict = {}

In [27]:
# Если ключи уникальны и можно просто обновлять словарь
for d in eval_dataset['paths']:
    if isinstance(d, dict):  
        merged_dict.update(d)

In [28]:
output_dir = os.path.join(dir_path_for_res, 'mapping_for_tifa_banch.json')
output_dir

'/home/student/kuzin/python_project/TextToSvg_MLCUP/data/Qwen/Qwen3-0.6B/mapping_for_tifa_banch.json'

In [29]:
if merged_dict:
    with open(output_dir, 'w', encoding='utf-8') as f:
        json.dump(dict(merged_dict), f, ensure_ascii=False, indent=4)

In [30]:
from timm.data import ImageNetInfo
#!!  pip install timm==1.0.13  версия на которой заводится

In [31]:
results = tifa_score_benchmark("mplug-large", "tifa/tifa_v1.0/tifa_v1.0_question_answers.json",output_dir)

# save the results
with open(os.path.join(dir_path_for_res ,'tifa_metrics.json'), "w") as f:
    json.dump(results, f, indent=4)

Loading mplug-large...


2025-06-18 20:39:56,316 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-06-18 20:40:00,265 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-06-18 20:40:00,729 - modelscope - INFO - initiate model from /home/student/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en
2025-06-18 20:40:00,729 - modelscope - INFO - initiate model from location /home/student/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en.
2025-06-18 20:40:00,731 - modelscope - INFO - initialize model from /home/student/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en
/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1894: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


load checkpoint from /home/student/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en/pytorch_model.bin
<All keys matched successfully>
Finish loading mplug-large
Using SBERT on GPU


  0%|          | 0/25829 [00:00<?, ?it/s]/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/student/kuzin/python_project/TextToSvg_MLCUP/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 25829/25829 [55:09<00:00,  7.80it/s] 


Average TIFA is 0.22188441806569073
